In [50]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
from ydata_profiling import ProfileReport,compare

import sys
sys.path.append('../../src/')
from namelist import *
from mask import polygon_to_mask, average_data

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

## 将网格文件转换为表格

In [21]:
month = "Sep"
years = [2014,2015,2016]

variables = ['SFC_TMP','AIR_TMP','RH','SOL_RAD',
             'WSPD10','WDIR10','PRES','CloudFRAC']
level = 1000 # units: hPa


In [ ]:
shp = gpd.read_file(shp_PRD_adm)
grid = xr.open_dataset(grid_d03)
lon = grid.LON.squeeze()
lat = grid.LAT.squeeze()

mask = polygon_to_mask(shp.geometry[0], lon, lat)
mask_da = xr.DataArray(mask, dims=('y','x'))

In [48]:
def nc_to_df(ds,variables,level,mask_da):
    df = {}
    for var in variables:
        data = ds[var].sel(level=level,method='nearest')
        masked_data = data.where(mask_da)
        mean = masked_data.mean(dim=('x','y'),skipna=True)
        df[var] = mean.to_dataframe().drop(columns=['level'])
    df_concat = pd.concat(df,axis=1)
    df_concat.columns = df_concat.columns.droplevel(0)
    
    return df_concat

## `ProfileReport`

In [49]:
reports = {}

for year in years:
    ds = xr.open_dataset(os.path.join(processed_dir, f"{month}_{year}/{month}_{year}_mcip.nc"))
    df = nc_to_df(ds,variables,level,mask_da)
    reports[year] = ProfileReport(df, title=f"{month}_{year}")
    

In [51]:
comparison = compare([reports[year] for year in years])

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

In [52]:
comparison.to_file('D:/Academic/Project/GRAD/Annually/data/profile_reports/Report_Low.html')

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]